In [1]:
#pip install --upgrade pip


In [2]:
#pip install html5lib

In [3]:
#pip install selenium

In [4]:
#pip install bs4

In [5]:
import pandas as pd
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import re

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By



In [11]:


driver = webdriver.Chrome()
driver.get("https://www.foxnews.com/category/shows/tucker-carlson-tonight/transcript")
button = driver.find_element(By.XPATH, "//div[@class='button load-more js-load-more']") 
x = 0
for x in range(144):
    button.click()    
    time.sleep(2)
    x+=1
    print(x)
page_source = driver.page_source # save the source to feed to beautiful soup 
print("saved")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
saved


In [12]:
soup = BeautifulSoup(page_source)


In [13]:
url_soup = soup.find_all('a', href=re.compile('/opinion/.*'))

In [14]:
url_list = []
prefix = 'https://www.foxnews.com/'
for link in url_soup: 
    suffix = re.findall('\"\/(.*?)\"', str(link))
    if suffix:
        url_list.append(prefix + suffix[0])
    

In [15]:
url_list  = [*set(url_list)] 

In [23]:
# # verifying for one instance
# url = url_list[100]
# #url = 'https://www.foxnews.com/opinion/tucker-carlson-faa-travel-pete-buttigieg'
# response = requests.get(url)
# soup = BeautifulSoup(response.content)
# title = soup.title.string

# a_tag = soup.find_all('a')
# for tag in a_tag:
#     tag.decompose()

# copy_tag = soup.find_all('p', class_ = "copyright")
# for tag in copy_tag:
#     tag.decompose()
    
# span_tag = soup.find_all('span')
# for tag in span_tag:
#     tag.decompose()
    
# bold_tag = soup.find_all('strong') 
# for tag in bold_tag:
#     tag.decompose()
    
# italic_tag=soup.find_all('i') 
# for tag in italic_tag:
#     tag.decompose()

# other_tag = soup.find_all('p', class_ ="dek")
# for tag in other_tag:
#     tag.decompose()
    
# other_tag2 = soup.find_all('p', class_ ="subscribed hide")
# for tag in other_tag2:
#     tag.decompose()
    
# other_tag3 = soup.find_all('p', class_ ="success hide") 
# for tag in other_tag3:
#     tag.decompose()



# final = soup.find_all('p')

# finaltext = ""
# for p in final:

#     try:
#         if len(p.text) > 1:
#             finaltext = finaltext + p.text + '\n'+ '\n'
#             finaltext = finaltext.replace('\xa0', ' ')

#     except:
#         continue
# print(finaltext)
len(url_list)

831

In [17]:

full_df = None
dfs = []

for n, url in enumerate(url_list):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    time = soup.find('time').text
    title = soup.title.string

    a_tag = soup.find_all('a')
    for tag in a_tag:
        tag.decompose()

    copy_tag = soup.find_all('p', class_ = "copyright")
    for tag in copy_tag:
        tag.decompose()

    span_tag = soup.find_all('span')
    for tag in span_tag:
        tag.decompose()

    bold_tag = soup.find_all('strong') 
    for tag in bold_tag:
        tag.decompose()

    italic_tag=soup.find_all('i') 
    for tag in italic_tag:
        tag.decompose()

    other_tag = soup.find_all('p', class_ ="dek")
    for tag in other_tag:
        tag.decompose()

    other_tag2 = soup.find_all('p', class_ ="subscribed hide")
    for tag in other_tag2:
        tag.decompose()

    other_tag3 = soup.find_all('p', class_ ="success hide") 
    for tag in other_tag3:
        tag.decompose()


    final = soup.find_all('p')
    # text=''
    # print(final)
    finaltext = ''
    for p in final:
        try:
            if len(p.text) > 1:
                finaltext = finaltext + p.text + '\n'+ '\n'
                finaltext = finaltext.replace('\xa0', ' ')

        except:
            continue
    
    data={'url': [url],'timestamp': [time], 'title': [title], 'transcript': [finaltext]}
    df = pd.DataFrame(data)
    dfs.append(df)

full_df = pd.concat(dfs).reset_index(drop=True)

In [18]:
full_df['timestamp'] = pd.to_datetime(full_df['timestamp'])

C:\Users\Owner\AppData\Local\Programs\Python\Python310\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
C:\Users\Owner\AppData\Local\Programs\Python\Python310\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [19]:
full_df=full_df.sort_values(by=['timestamp']).reset_index(drop=True)

In [21]:
full_df.to_csv('data/Tucker_transcripts.tsv', sep="\t")

In [24]:
len(full_df)

831

In [25]:
full_df

,url,timestamp,title,transcript
0,https://www.foxnews.com/opinion/tucker-carlson...,2018-11-27 08:39:00,Tucker Carlson: Socialism with open borders is...,Democrats want to eliminate restrictions on im...
1,https://www.foxnews.com/opinion/tucker-carlson...,2018-11-28 07:16:00,"Tucker Carlson: For the crime of forgetting, J...",Jerome Corsi addresses charges against him in ...
2,https://www.foxnews.com/opinion/tucker-carlson...,2018-11-29 11:12:00,Tucker Carlson: The word 'Russia' eliminates a...,"On Wednesday, Tucker Carlson exposes the flaws..."
3,https://www.foxnews.com/opinion/tucker-carlson...,2018-11-30 09:21:00,Tucker Carlson: The Mueller probe and what we ...,Michael Cohen pleads guilty to lying to Congre...
4,https://www.foxnews.com/opinion/tucker-carlson...,2018-12-04 08:08:00,Tucker Carlson: The Mueller probe continues to...,"On Monday, Tucker Carlson talks to Jerome Cors..."
...,...,...,...,...
826,https://www.foxnews.com/opinion/tucker-carlson...,2023-03-10 22:41:00,TUCKER CARLSON: Silicon Valley Bank has gone c...,Fox News host Tucker Carlson breaks down the c...
827,https://www.foxnews.com/opinion/tucker-carlson...,2023-03-13 23:06:00,TUCKER CARLSON: The Biden administration sees ...,Fox News host Tucker Carlson takes a closer lo...
828,https://www.foxnews.com/opinion/tucker-carlson...,2023-03-14 23:57:00,TUCKER CARLSON: We're getting moral lectures f...,Fox News host Tucker Carlson calls out wokenes...
829,https://www.foxnews.com/opinion/tucker-carlson...,2023-03-15 23:22:00,TUCKER CARLSON: You're going to get a hot war ...,Fox News host Tucker Carlson calls out the Bid...
